In [1]:
import numpy as np
import pandas as pd

## EDA

In [41]:
aorDf = pd.read_csv("../data/accidents_on_road.csv").drop(columns=["result_id"])
aorDf['objectid'] = aorDf['objectid'].astype(int).astype(str)
print(aorDf.shape)
aorDf.head()

(133611, 2)


,report_id,objectid
0,171111,8708
1,192016,2649
2,192016,8393
3,192016,8392
4,192016,72097


In [56]:
aorDf.objectid

0           8708
1           2649
2           8393
3           8392
4          72097
           ...  
133606     53029
133607    100920
133608     13657
133609    153481
133610     74916
Name: objectid, Length: 133611, dtype: object

In [42]:
aorDf.to_csv("../data/accidents_on_road_revised.csv", index=False)

In [46]:
aDf = pd.read_csv("../data/accidents.csv").drop(columns=["result_id"])
print(aDf.columns)
aDf = aDf[['report_id','date_time','violation_section','violation_type','charge_desc',
           'injured','killed','hit_run_lvl']]
for c in aDf.columns:
    if c in ['injured','killed']: aDf[c] = aDf[c].astype(int)
    else: aDf[c] = aDf[c].astype(str)
print(aDf.shape)
aDf.head()

Index(['report_id', 'date_time', 'police_beat', 'address_no_primary',
       'address_pd_primary', 'address_road_primary', 'address_sfx_primary',
       'address_pd_intersecting', 'address_name_intersecting',
       'address_sfx_intersecting', 'violation_section', 'violation_type',
       'charge_desc', 'injured', 'killed', 'hit_run_lvl', 'x', 'y'],
      dtype='object')
(63970, 8)


,report_id,date_time,violation_section,violation_type,charge_desc,injured,killed,hit_run_lvl
0,171111,2015-01-14 20:00:00,MISC-HAZ,VC,MISCELLANEOUS HAZARDOUS VIOLATIONS OF THE VEHI...,0,0,MISDEMEANOR
1,192016,2015-03-19 12:00:00,MISC-HAZ,VC,MISCELLANEOUS HAZARDOUS VIOLATIONS OF THE VEHI...,0,0,MISDEMEANOR
2,190012,2015-03-24 03:05:00,22107,VC,TURNING MOVEMENTS AND REQUIRED SIGNALS,2,0,nan
3,191866,2015-03-27 23:56:00,22107,VC,TURNING MOVEMENTS AND REQUIRED SIGNALS,1,0,nan
4,185207,2015-07-06 11:45:00,20002(A),VC,HIT AND RUN,0,0,MISDEMEANOR


In [47]:
aDf.to_csv("../data/accidents_revised.csv", index=False)

In [19]:
gtdrDf = pd.read_csv("../data/get-it-done-reports.csv").drop(columns=["Unnamed: 0"])
gtdrDf = gtdrDf.fillna(0)
print(gtdrDf.columns)
for c in gtdrDf.columns:
    if c=='case_age_days': 
        gtdrDf[c] = gtdrDf[c].astype(int)
        continue
    if isinstance(gtdrDf[c][0], float): gtdrDf[c] = gtdrDf[c].astype(int).astype(str)
    gtdrDf[c] = gtdrDf[c].astype(str)
    gtdrDf[c] = gtdrDf[c].replace(['0'], '')
gtdrDf = gtdrDf[['service_request_id','date_requested','case_age_days','service_name',
                 'service_name_detail','status','street_address','council_district',
                 'case_origin','public_description']]
print(gtdrDf.shape)
gtdrDf.head()

Index(['service_request_id', 'service_request_parent_id',
       'sap_notification_number', 'date_requested', 'case_age_days',
       'case_record_type', 'service_name', 'service_name_detail',
       'date_closed', 'status', 'lat', 'lng', 'street_address', 'zipcode',
       'council_district', 'comm_plan_code', 'comm_plan_name', 'park_name',
       'case_origin', 'referred', 'iamfloc', 'floc', 'public_description'],
      dtype='object')
(62819, 10)


,service_request_id,date_requested,case_age_days,service_name,service_name_detail,status,street_address,council_district,case_origin,public_description
0,100763,2016-08-20T14:46:00,2470,Street Sweeping,,In Process,"5135 University Avenue, San Diego, CA 92105, U...",9,Web,A) The storm drain channel south of 5135 Univ...
1,100777,2016-08-20T15:48:00,2470,Sidewalk Repair Issue,SIDEWALK MINOR REHAB CONTRACT,In Process,"4300-4324 37th St, San Diego, CA 92105, USA",9,Web,Curb in rubble
2,100985,2016-08-22T10:04:00,2468,Stormwater,DRAIN HEADWALL,In Process,"6175 NANCY RIDGE DR, 92121",6,Phone,HILLSIDE ERODING - POSSIBLE BROKEN DRAIN
3,101169,2016-08-23T08:30:00,2467,Pavement Maintenance,DAMAGED CURB,In Process,"2950-52 National Avenue, San Diego",8,Web,Large sections of curb broke off and have been...
4,101707,2016-08-25T15:01:00,2465,Pavement Maintenance,DAMAGED CURB,In Process,"2220 Ocean View Boulevard, San Diego, CA, Unit...",8,Web,the curb in front of 2220 Ocean View Blvd. nee...


In [21]:
# gtdrDf.to_csv("../data/get-it-done-reports_revised.csv", index=False)

In [23]:
rorDf = pd.read_csv("../data/reports_on_road.csv").drop(columns=["Unnamed: 0"])
for c in rorDf.columns:
    rorDf[c] = rorDf[c].astype(int).astype(str)
print(rorDf.shape)
rorDf.head()

(73535, 2)


,service_request_id,objectid
0,100777,69508
1,100985,20182
2,101798,93638
3,101799,93638
4,101870,22009


In [25]:
# rorDf.to_csv("../data/reports_on_road_revised.csv", index=False)

In [35]:
rDf = pd.read_csv("../data/roads.csv").drop(columns=["Unnamed: 0"])
print(rDf.columns)
rDf = rDf[['objectid','roadsegid','roadid','rd20full','speed',
           'oneway','firedriv','postdate','addsegdt']]
for c in rDf:
    if c=='speed': rDf[c] = rDf[c].astype(int)
    else:
        rDf[c] = rDf[c].astype(str)
print(rDf.shape)
rDf.head()

Index(['objectid', 'roadsegid', 'roadid', 'fnode', 'tnode', 'rd20full',
       'rd30full', 'speed', 'oneway', 'firedriv', 'funclass', 'segclass',
       'segstat', 'dedstat', 'subdivid', 'rightway', 'carto', 'obmh',
       'postdate', 'addsegdt', 'min_addr', 'max_addr', 'f_level', 't_level',
       'l_block', 'r_block', 'l_tract', 'r_tract', 'l_beat', 'r_beat',
       'l_psblock', 'r_psblock', 'wgs_from_x', 'wgs_from_y', 'wgs_to_x',
       'wgs_to_y', 'nad_from_x', 'nad_from_y', 'nad_to_x', 'nad_to_y',
       'rd20pred', 'rd20name', 'rd20sfx', 'rd30pred', 'rd30name', 'rd30sfx'],
      dtype='object')
(57152, 9)


,objectid,roadsegid,roadid,rd20full,speed,oneway,firedriv,postdate,addsegdt
0,17,2014,2225,08TH AV,20,T,Y,2020-02-04,1980-04-01
1,18,2027,2225,08TH AV,20,T,Y,2019-01-16,1980-04-01
2,19,2029,2225,08TH AV,20,T,Y,2020-02-04,1980-04-01
3,20,2031,2225,08TH AV,20,T,Y,2020-02-04,1980-04-01
4,21,2034,2225,08TH AV,20,B,Y,2019-09-03,1980-04-01


In [36]:
# rDf.to_csv("../data/roads_revised.csv", index=False)

## Connect to Postgre and Write Data

In [70]:
import psycopg2
from tqdm import tqdm

# Connect to the Postgres container using the container name as the hostname
conn = psycopg2.connect(
    dbname="my_database",
    user="postgres",
    password="mysecretpassword",
    host="localhost",
    port="5433"
)
cursor = conn.cursor()

In [71]:
query = "SELECT tablename FROM pg_catalog.pg_tables WHERE schemaname='public';"
cursor.execute(query)
table_names = cursor.fetchall()
for table_name in table_names:
    print(table_name[0])

In [67]:
# delete all previous tables
for table_name in table_names:
    drop_query = """DROP TABLE IF EXISTS {} CASCADE;""".format(table_name[0])
    cursor.execute(drop_query)
conn.commit()

In [72]:
tempDf = pd.read_csv("../data/accidents_on_road_revised.csv")
tempDf.head()

,report_id,objectid
0,171111,8708
1,192016,2649
2,192016,8393
3,192016,8392
4,192016,72097


In [59]:
# aorDf.objectid.apply(lambda x: len(x)).max()

In [73]:
create_table_query = \
    """
    CREATE TABLE IF NOT EXISTS accidents (
        report_id VARCHAR(10),
        objectid VARCHAR(10)
    )
    """
cursor.execute(create_table_query)
conn.commit()

In [74]:
with open("../data/accidents_on_road_revised.csv", "r") as f:
    next(f)
    cursor.copy_from(f, "accidents", sep=",", null="")

In [75]:
select_query = \
    """
    SELECT * FROM accidents
    """
cursor.execute(select_query)
rows = cursor.fetchall()
for r in rows:
    print(r)
    break

('171111', '8708')


In [64]:
cursor.close()
conn.close()